In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tiny_shakespear.txt

--2023-12-11 19:36:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tiny_shakespear.txt’

tiny_shakespear.txt 100%[===================>]   1.06M  4.89MB/s    in 0.2s    

2023-12-11 19:36:47 (4.89 MB/s) - ‘tiny_shakespear.txt’ saved [1115394/1115394]



In [4]:
with open("tiny_shakespear.txt", 'r', encoding='utf-8') as handle:
    text = handle.read()

In [5]:
print(f"Length of the dataset: {len(text)}")

Length of the dataset: 1115394


In [6]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [7]:
characters = sorted(
    list(
        set(
            text
        )
    )
)
vocabulary_size = len(characters)
print(f"All available characters: {''.join(characters)}")
print(f"Their count: {vocabulary_size}")

All available characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Their count: 65


In [8]:
string_to_int = {char: index for index, char in enumerate(characters)}
int_to_string = {index: char for index, char in enumerate(characters)}

encode = lambda string: [string_to_int[character] for character in string]
decode = lambda list_of_integers: ''.join([int_to_string[integer] for integer in list_of_integers])

print(f"{encode('hi mom!')=}")
print(f"{decode(encode('hi mom!'))=}")

encode('hi mom!')=[46, 47, 1, 51, 53, 51, 2]
decode(encode('hi mom!'))='hi mom!'


In [9]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(f"{data.shape=}, {data.dtype=}")
print(f"{data[:200]=}")

data.shape=torch.Size([1115394]), data.dtype=torch.int64
data[:200]=tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [10]:
n = int (0.9 * len(data))

train_data = data[:n]
validation_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1: block_size + 1]

for token in range(block_size):
    context = x[: token + 1]
    target = y[token]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [13]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else validation_data
    
    ix = torch.randint(
        len(data) - block_size, 
        (batch_size,)
    )
    x = torch.stack([
        data[i: i + block_size]
        for i in ix
    ])
    y = torch.stack([
        data[i + 1: i + block_size + 1]
        for i in ix
    ])
    return x, y
    
xb, yb = get_batch('train')
print(f'INPUTS {xb.shape=}\n {xb=}\n')
print(f'TARGETS {yb.shape=}\n {yb=}')

for batch in range(batch_size):
    for token in range(block_size):
        context = xb[batch, : token + 1]
        target = yb[batch, token]
        print(f"when input is {context.tolist()} the target: {target}")

INPUTS xb.shape=torch.Size([4, 8])
 xb=tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

TARGETS yb.shape=torch.Size([4, 8])
 yb=tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
w

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)
                                                 
    def forward(self, idx, targets = None):    
        logits = self.token_embedding_table(idx) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)            
            loss = F.cross_entropy(logits, targets)       
            
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx
        
model = BigramLanguageModel(vocabulary_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(
    model.generate(
        torch.zeros((1, 1), dtype=torch.long),
        max_new_tokens=100
    )[0].tolist()
))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [16]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5727508068084717


In [17]:
print(decode(
    model.generate(
        torch.zeros((1, 1), dtype=torch.long),
        max_new_tokens=200
    )[0].tolist()
))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo 


# Self-attention

In [18]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # Batch, Time, Channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# version 1
x_bag_of_words = torch.zeros((B, T, C))
for batchh in range(B):
    for timee in range(T):
        x_previous = x[batchh, : timee + 1]
        x_bag_of_words[batchh, timee] = torch.mean(x_previous, 0)

In [20]:
# version 2
tril = torch.tril(torch.ones(T, T))
weights = tril / tril.sum(1, keepdim=True)
weights  # a
x_bag_of_words2 = weights @ x  # (B, T, T) @ (B, T, C) ----> (B, T , C)
torch.allclose(x_bag_of_words, x_bag_of_words2)

False

In [21]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [22]:
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril == 0, float('-inf'))
# weights = F.softmax(weights, dim=-1)
weights

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [23]:
# version 3
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)
x_bag_of_words3 = weights @ x
torch.allclose(x_bag_of_words, x_bag_of_words3)

False

In [24]:
x[0], x_bag_of_words[0], x_bag_of_words2[0], x_bag_of_words3[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [25]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(f"{a=}")
print(f"{b=}")
print(f"{c=}")

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


## Single head of self-attention

In [26]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)

tril = torch.tril(torch.ones(T, T))

weights = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
weights = weights.masked_fill(tril == 0, float('-inf'))  # decoder
# weights = weights.masked_fill(tril == 0, float('-inf'))  # encoder
weights = F.softmax(weights, dim=1)

v = value(x)
output = weights @ v
output.shape

torch.Size([4, 8, 16])

In [27]:
weights[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)

In [28]:
_k = torch.randn(B, T, head_size)
_q = torch.randn(B, T, head_size)
_wei = _q @ _k.transpose(-2, -1) * head_size**-0.5

In [29]:
_k.var()

tensor(1.0449)

In [30]:
_q.var()

tensor(1.0700)

In [31]:
_wei.var()

tensor(1.0918)

In [32]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [33]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [34]:
class BatchNorm1d:
    
    def __init__(self, dimensions, epsilon=1e-5, momentum=0.1) -> None:
        self.epsilon = epsilon
        self.momentum = momentum
        self.training = True
        
        # parameters (trained with back propagation)
        self.gamma = torch.ones(dimensions)
        self.beta = torch.zeros(dimensions)
        
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dimensions)
        self.running_variance = torch.ones(dimensions)
        
    def __call__(self, x):
        if self.training:
            x_mean = x.mean(0, keepdim=True)  # batch mean
            x_var = x.var(0, keepdim=True)  # batch variance
        else:
            x_mean = self.running_mean
            x_var = self.running_variance
            
        # calculate the forward pass        
        xhat = (x - x_mean) / torch.sqrt(x_var + self.epsilon)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * x_mean
                self.running_variance = (1 - self.momentum) * self.running_variance + self.momentum * x_var
        
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)  # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [35]:
x[:,0].mean(), x[:,0].std()

(tensor(7.4506e-09), tensor(1.0000))

In [37]:
x[0,:].mean(), x[0,:].std()

(tensor(0.0411), tensor(1.0431))

In [44]:
class LayerNorm:
    
    def __init__(self, dimensions, epsilon=1e-5, momentum=0.1) -> None:
        self.epsilon = epsilon
        # parameters (trained with back propagation)
        self.gamma = torch.ones(dimensions)
        self.beta = torch.zeros(dimensions)
        
    def __call__(self, x):
        x_mean = x.mean(1, keepdim=True)  # batch mean
        x_var = x.var(1, keepdim=True)  # batch variance
        # calculate the forward pass        
        xhat = (x - x_mean) / torch.sqrt(x_var + self.epsilon)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
torch.manual_seed(1337)
module = LayerNorm(100)
x = torch.randn(32, 100)  # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [45]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [46]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))